# Scan Models
**Learning about the different scan models and their parameters, and when to use them.**

The scan model is one of the core component that defines a ptychography experiment, it creates different data structures such as [Views](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.View), [Containers](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.Container) and [Storages](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.Storage). It also creates the [PODs](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.POD) which link the containers (and views) of probe, object, exit wave and diffraction data. PtyPy offers a range of different scan models

- The most commonly used scan model is `Full` and should work in most cases, but is not very performant for large datasets (>10k frames)

- `GradFull` is a variation of `Full` that does not create separate exit containers for each POD - it therefore has a much lower memory footprint while remaining compatible with the `ePIE` and `ML` engines (not compatible with `DM` and similar engines). 

- `BlockFull` is another variation of `Full` geared towards large data and accelerated engines. `BlockFull` requires `p.frames_per_block` to be set to a reasonable value. More about scan models for GPU acceleration in the tutorial about [the block model](../Processing_Large_Datasets/the_block_model.ipynb).

- `Vanilla` is a lightweight scan model that can be useful for simple simulations.

All the `Full` - flavoured models share the same set of [parameters](https://ptycho.github.io/ptypy/rst/parameters.html#scan-full), while `Vanilla` has a much smaller sub-set of [parameters](https://ptycho.github.io/ptypy/rst/parameters.html#scan-vanilla).


Let's look at some of these parameters in a bit more detail. When we define a scan in the parameter tree, we can give it a name (e.g. `MF`) and specify the scan model to be used (e.g. `Full`)
```python
p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"
```

Next, we can specify the initial illumination to be a circular aperture with a 3-micron diameter and a 2-pixel wide edge
```python
p.scans.MF.illumination = u.Param()
p.scans.MF.illumination.aperture = u.Param()
p.scans.MF.illumination.aperture.form = "circ"
p.scans.MF.illumination.aperture.size = 3e-6
p.scans.MF.illumination.aperture.offset = 0
p.scans.MF.illumination.aperture.edge = 2
```

We should also define the number of photons of the incident illumination. Setting `photons` to `None` will estimate this value from the statistics of the data
```python
p.scans.MF.illumination.photons = None
```

For the initial object we can just use an empty image filled with ones
```python
p.scans.MF.sample = u.param()
p.scans.MF.sample.fill = 1
p.scans.MF.sample.process = None
```

PtyPy can reconstruct data from ptychography experiments that are collected in either `farfield` or `nearfield` geometry
```python
p.scans.MF.propagation = "farfield"
```

And finally, we need to specify which data to load into the scan model. Here we can use the same `MoonFlowerScan` simulation as in previous examples
```python
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.
```

`````{admonition} Exercise 
:class: attention
Modify the <code>illumination</code> and <code>object</code> parameters in the example below and observe how it affects the initial state and final state of the reconstruction by using different levels in <code>Ptycho(...)</code> and <code>ptypy.utils.plot_client.figure_from_ptycho</code> for plotting. A change in the initial state of illumination and/or object might affect the convergence and/or quality of the reconstruction.
`````

---

In [ ]:
import ptypy
import ptypy.utils as u

p = u.Param()
p.verbose_level = "interactive"
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Live-plotting
p.io.autoplot = u.Param()
p.io.autoplot.active=True
p.io.autoplot.threaded = False
p.io.autoplot.layout = "jupyter"
p.io.autoplot.interval = 10

# Define the scan model
p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"

# Define initial illumination
p.scans.MF.illumination = u.Param()
p.scans.MF.illumination.aperture = u.Param()
p.scans.MF.illumination.aperture.form = "circ"
p.scans.MF.illumination.aperture.size = 3e-6
p.scans.MF.illumination.aperture.offset = 0
p.scans.MF.illumination.aperture.edge = 2

# Define initial object
p.scans.MF.sample = u.Param()
p.scans.MF.sample.fill = 1
p.scans.MF.sample.process = None

# Set the nr. of photons in the initial illumination
# None will use data to estimate this value
p.scans.MF.illumination.photons = None

# Farfield or Nearfield geometry
p.scans.MF.propagation = "farfield"

# Data loader / simulator 
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

p.engines = u.Param()
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM"
p.engines.engine00.numiter = 80

P = ptypy.core.Ptycho(p,level=4)

In [ ]:
fig = ptypy.utils.plot_client.figure_from_ptycho(P)